In [1]:
import os
from datetime import datetime
import librosa
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
from scipy.stats import skew

In [2]:
PATH = os.getcwd() + '/audio_dataset/'
SAVING_PATH = os.getcwd() + '/Chroma_STFT_npy_dataset/'
print(PATH)
print(SAVING_PATH)

d:\Study\Code\Python\Sound_Classification_Bee_Qeen_Queenless/audio_dataset/
d:\Study\Code\Python\Sound_Classification_Bee_Qeen_Queenless/Chroma_STFT_npy_dataset/


In [3]:
NAV_DIR = ["train/Queen/", "train/Queenless/", "test/Queen/", "test/Queenless/", "val/Queen/", "val/Queenless/"]

In [4]:
def read_16k_mono(path):
    y, sr = librosa.load(path, sr=16000, mono=True)
    return y, sr

In [11]:
def stft(y, n_fft, hop_length, win_length, window, center=True):
    feature = librosa.feature.chroma_stft(y=y, sr=16000, n_fft=n_fft, hop_length=hop_length, win_length=win_length, window=window, center=center)
    # print(feature.shape)
    feature_mean = np.mean(np.abs(feature).T,axis=0) 
    feature_std=np.std(np.abs(feature).T,axis=0)
    # feature_min=np.min(np.abs(feature).T,axis=0)
    # feature_max=np.max(np.abs(feature).T,axis=0)
    # feature_skew = skew(np.abs(feature).T,axis=0)
    # feature_median = np.median(np.abs(feature).T,axis=0)
    # feature = np.hstack((feature_mean,feature_std,feature_min,feature_max,feature_skew,feature_median))
    feature = np.hstack((feature_mean,feature_std))
    return feature

In [6]:
FILE_PATH = os.listdir(PATH+ NAV_DIR[0])[0]
y, sr = read_16k_mono(PATH + NAV_DIR[0] + FILE_PATH)
feature = stft(y, n_fft=1024, hop_length=None, win_length=None, window = "hamming")
feature = feature.reshape(feature.shape[0], 1)
print("STFT feature shape: ", feature.shape)

STFT feature shape:  (72, 1)


- time_frame = (TIME*SR)//HOP_LENGTH
- pitch = 12
- in stft
- frequerency_bin = (win_length//2)+1

**Why axis = 1 but still keep 126 columns??**

In [7]:
def data_extract_stft(path):
    data = []
    print("Extracting data from " + path)
    for file in os.listdir(path):
        if file.endswith(".wav"):
            y, sr = read_16k_mono(path + file)
            feature = stft(y, n_fft=1024, hop_length=None, win_length=None, window = "hann")
            data.append(feature)
    print(f"Extracted {len(data)} files from {path}")
    return data

In [8]:
DATA = []
time0 = datetime.now()
for SUB_PATH in NAV_DIR:
    sub_data = data_extract_stft(PATH + SUB_PATH)
    DATA.append(sub_data)
time1 = datetime.now()
extract_time = (time1 - time0).seconds / 60
print('Extract time: %.2f minutes' % extract_time)

Extracting data from d:\Study\Code\Python\Sound_Classification_Bee_Qeen_Queenless/audio_dataset/train/Queen/


c:\Users\minht\anaconda3\envs\gpu\lib\site-packages\librosa\core\pitch.py:102: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


Extracted 7000 files from d:\Study\Code\Python\Sound_Classification_Bee_Qeen_Queenless/audio_dataset/train/Queen/
Extracting data from d:\Study\Code\Python\Sound_Classification_Bee_Qeen_Queenless/audio_dataset/train/Queenless/
Extracted 7000 files from d:\Study\Code\Python\Sound_Classification_Bee_Qeen_Queenless/audio_dataset/train/Queenless/
Extracting data from d:\Study\Code\Python\Sound_Classification_Bee_Qeen_Queenless/audio_dataset/test/Queen/
Extracted 2000 files from d:\Study\Code\Python\Sound_Classification_Bee_Qeen_Queenless/audio_dataset/test/Queen/
Extracting data from d:\Study\Code\Python\Sound_Classification_Bee_Qeen_Queenless/audio_dataset/test/Queenless/
Extracted 2000 files from d:\Study\Code\Python\Sound_Classification_Bee_Qeen_Queenless/audio_dataset/test/Queenless/
Extracting data from d:\Study\Code\Python\Sound_Classification_Bee_Qeen_Queenless/audio_dataset/val/Queen/
Extracted 1000 files from d:\Study\Code\Python\Sound_Classification_Bee_Qeen_Queenless/audio_datas

In [9]:
print(np.array(DATA[0]).shape)

(7000, 72)


In [10]:
for index in range(len(DATA)):
    np.save(SAVING_PATH + NAV_DIR[index] + NAV_DIR[index].split('/')[-2] + '_Choma_STFT.npy', np.array(DATA[index]))
    print('Saving ' + NAV_DIR[index].split('/')[-2] + '_Choma_STFT.npy')

Saving Queen_Choma_STFT.npy
Saving Queenless_Choma_STFT.npy
Saving Queen_Choma_STFT.npy
Saving Queenless_Choma_STFT.npy
Saving Queen_Choma_STFT.npy
Saving Queenless_Choma_STFT.npy
